# figure 5 vlz

- kernel: r_env, R 4.1.3
- date: 2024-02-05
- desc: part of fig4-new & fig4

## load

In [6]:
library(tidyverse)
library(logging)
library(ggpubr)
library(ggsci)
library(patchwork)

source('../../stage2/scripts/r_funcs.r')

theme_set(theme_pubr())
logging::basicConfig()
options(warn = -1)

outdir <- '../figures/fig5'
create_dir(outdir)

In [7]:
f_pat_gp <- '../tables/patient_info_v2.tsv'
gp <- 'response'
gp_lvls$response <- c('R', 'PR')
gp_comp_map$response <- list(
  c('R-pre', 'R-post'),
  c('PR-pre', 'PR-post'),
  c('R-pre', 'PR-pre'),
  c('R-post', 'PR-post')
)
gp_comp_diff_map$response <- list(c('R', 'PR'))
gp_comp_map_pre$response <- list(c('R-pre', 'PR-pre'))
comb_order <- c('R-pre', 'R-post', 'PR-pre', 'PR-post')

In [8]:
celltype_map <- list(
    'T_MKI67' = c('T_Prolif')
)

cell_state_map <- list(
    'cytotoxic' = c('CD8_ANXA1', 'CD8_CCL5', 'CD8_CX3CR1', 'CD8_FOS', 'CD8_GZMK', 'CD8_KLRB1'),
    'exhausted' = c('CD4_CXCL13', 'CD8_CXCL13', 'CD8_TYMS', 'T_MKI67', 'Treg_TNFRSF4'),
    'dying' = c('T_Mito'),
    'others' = c('CD4_KLRB1', 'CD8_CD74', 'CD8_IFIT1', 'T_IL7R', 'T_Ribo', 'Treg_LTB')
)
cell_state_order <- c('cytotoxic', 'exhausted', 'dying', 'others')
cell_state_color <- c('exhausted' = '#023fa5', 'cytotoxic' = 'red', 
                      'dying' = '#bb7784', 'others' = '#ff9639')

## 5a: pseudo forest plot

### functions

In [4]:
cal_stat <- function(x, y) {
    t_obj <- t.test(x = x, y = y)
    res <- data.frame(
        mean_diff = t_obj$estimate[1] - t_obj$estimate[2],
        conf_int_upper = t_obj$conf.int[2],
        conf_int_lower = t_obj$conf.int[1],
        stderr = t_obj$stderr,
        pval = t_obj$p.value
    )
    return(res)
}

### expansion

no NK cells, paird patients, delta (good-poor) of post-pre delta expand in subtype

In [7]:
f_expand_diff <- '../../stage4/a03_tcr/expansion/diff_expandPct_min10.csv'

# post-pre delta
df_delta <- read_csv(f_expand_diff, show_col_types = F) %>% 
    filter(!grepl('^NK', subtype)) %>%
    select(patient, subtype, Baseline, Treat, pct_expand_diff) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')

for (nm in names(celltype_map)) {
    df_delta$subtype[df_delta$subtype %in% celltype_map[[nm]]] <- nm
}
loginfo('%g records, %g patients, %g celltypes', nrow(df_delta), length(unique(df_delta$patient)), length(unique(df_delta$subtype)))

2024-02-05 17:04:58 INFO::these clinial info will be added: response
2024-02-05 17:04:58 INFO::411 records, 28 patients, 18 celltypes


In [9]:
df_delta %>% write_tsv(str_glue('{outdir}/fig5a-expand_in_subtype-post_pre_delta.tsv'))

In [10]:
# cal stat by subtype
res <- do.call(rbind.data.frame, lapply(X = unique(df_delta$subtype), FUN = function(ctype) {
    cal_stat(x = filter(df_delta, subtype == ctype, .data[[gp]] == 'R') %>% pull(pct_expand_diff),
             y = filter(df_delta, subtype == ctype, .data[[gp]] == 'PR') %>% pull(pct_expand_diff)) %>%
    mutate(subtype = ctype)
}))
# add cell state info
res$cell_state <- 'others'
for (nm in names(cell_state_map)){
    res$cell_state[res$subtype %in% cell_state_map[[nm]]] <- nm
}
loginfo('%g records, %g celltypes', nrow(res), length(unique(res$subtype)))
res %>% write_tsv(str_glue('{outdir}/fig5a-expand_in_subtype-delta_delta-stat.tsv'))

2024-02-05 17:05:47 INFO::18 records, 18 celltypes


In [14]:
p <- read_tsv(str_glue('{outdir}/fig5a-expand_in_subtype-delta_delta-stat.tsv'), show_col_types = F) %>%
    mutate(subtype = factor(subtype, levels = unlist(cell_state_map)), 
           cell_state = factor(cell_state, levels = names(cell_state_map))) %>%
    ggplot(aes(x = mean_diff, y = subtype, color = cell_state, label = subtype)) +
        geom_pointrange(aes(xmin = conf_int_lower, xmax = conf_int_upper), linewidth = 1, size = 1.5, shape = 18) +
        geom_vline(xintercept = 0, color = 'black') +
        geom_text(color = 'black', size = 3.5, nudge_x = -5, nudge_y = 0.3) +
        scale_color_manual(values = cell_state_color) +
        guides(color = guide_legend(ncol = 2, title.position = 'top')) +
        labs(x = bquote(atop(Delta~'expanded cell percent', '(post - pre)')), y = '', color = 'Cell state') + 
        theme(axis.line.y = element_blank(), axis.text.y = element_blank(), axis.ticks.y = element_blank())
ggsave(filename = str_glue('{outdir}/fig5a-pseudo_forest-expand_in_subtype-delta_delta.pdf'), plot = p, width = 2.5, height = 7)

## 5b: boxplot for CD8_CX3CR1

- diversity, diff of expand, expansion percent

In [1]:
f_expand <- '../../stage4/a03_tcr/expansion/expand_cell_pct.csv'
f_expand_diff <- '../figures/fig5/fig5a-expand_in_subtype-post_pre_delta.tsv'  # this is based on min10 cells results
f_diversity <- '../../stage4/a03_tcr/diversity/shannon_way2_min10.csv'

min_cell_per_sample_subtype <- 10

### expand in cluster

In [26]:
# process
df <- read_csv(f_expand, show_col_types = F) %>%
    filter(subtype == 'CD8_CX3CR1') %>%
    filter(n_cell_per_sample_subtype >= min_cell_per_sample_subtype) %>%
    select(patient, sample_type, n_expand_cell_per_sample_stype, n_cell_per_sample_subtype,
           pct_in_cluster = pct_by_subtype, pct_in_sample = pct_by_sample) %>% 
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post'))
loginfo('%g samples for CD8_CX3CR1', nrow(df))
df %>% write_tsv(str_glue('{outdir}/fig5b-expand_pct-cd8_cx3cr1-min{min_cell_per_sample_subtype}.tsv'))

2024-02-05 17:12:18 INFO::these clinial info will be added: response
2024-02-05 17:12:18 INFO::73 samples for CD8_CX3CR1


In [27]:
# plot
p <- read_tsv(str_glue('{outdir}/fig5b-expand_pct-cd8_cx3cr1-min{min_cell_per_sample_subtype}.tsv'), show_col_types = F) %>% 
    filter(!is.na(.data[[gp]])) %>% 
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'pct_in_cluster', pt_fill = gp, facet_by = NULL,
                      xorder = comb_order, fill_order = gp_lvls[[gp]], xangle = 60) +
    stat_compare_means(comparisons = gp_comp_map[[gp]]) +
    labs(y = 'Expanded cell percent in CD8_CX3CR1', fill = 'Response')
ggsave(filename = str_glue('{outdir}/fig5b-box_expand-in_cluster-cx3cr1.pdf'), plot = p, width = 3, height = 4.5)

### expand diff in cluster

In [ ]:
p <- read_tsv(f_expand_diff, show_col_types = F) %>% 
    filter(subtype == 'CD8_CX3CR1') %>%
    filter(!is.na(.data[[gp]])) %>% 
    cell_comp_boxplot(x = gp, y = 'pct_expand_diff', pt_fill = gp, pair_by = NULL, facet_by = NULL,
                      xorder = gp_lvls[[gp]], fill_order = gp_lvls[[gp]], xangle = 60) +
    stat_compare_means(comparisons = gp_comp_diff_map[[gp]]) +
    labs(y = bquote(atop(Delta~'expanded cell percent in CD8_CX3CR1', '(post - pre)')), fill = 'Response') +
    theme(legend.justification = c(1, 0))
ggsave(filename = str_glue('{outdir}/fig5b-box_expand_diff-cx3cr1.pdf'), plot = p, width = 3, height = 4.5)

### diversity: shannon index

In [29]:
# process
df <- read_csv(f_diversity, show_col_types = F) %>%
    filter(subtype == 'CD8_CX3CR1') %>%
    select(patient, sample_type, shannon) %>% 
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post'))

loginfo('%g samples for CD8_CX3CR1', nrow(df))
df %>% write_tsv(str_glue('{outdir}/fig5b-diversity-cd8_cx3cr1-min{min_cell_per_sample_subtype}.tsv'))

2024-02-05 17:13:39 INFO::these clinial info will be added: response
2024-02-05 17:13:39 INFO::73 samples for CD8_CX3CR1


In [30]:
p <- read_tsv(str_glue('{outdir}/fig5b-diversity-cd8_cx3cr1-min{min_cell_per_sample_subtype}.tsv'), show_col_types = F) %>% 
    filter(!is.na(.data[[gp]])) %>% 
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'shannon', pt_fill = gp, facet_by = NULL,
                      xorder = comb_order, fill_order = gp_lvls[[gp]], xangle = 45) +
    stat_compare_means(comparisons = gp_comp_map[[gp]]) +
    labs(y = 'Shannon index', fill = 'Response')
ggsave(filename = str_glue('{outdir}/fig5b-box_diversity-cx3cr1.pdf'), plot = p, width = 3, height = 4.5)

## 5c: cyto/exhau score by A/B share/specific clones

- group cells by TCR sharing type: A/B-share/specific (A = CD8_CX3CR1)
- cyto/exhau summarise at cell level by TCR sharing type & sample & functional group

In [41]:
f_info <- '../../stage4/a05_clone_share/a_b_share_old/clone_cat_to_CD8_CX3CR1_per_cell.csv'
comp_ls <- list(
    c('A-specific', 'A-shared'),
    c('A-shared', 'B-shared'),
    c('B-specific', 'B-shared'),
    c('A-specific', 'B-shared')
)
group_order <- c('A-specific', 'A-shared', 'B-shared', 'B-specific')

In [42]:
# process
clone_info <- read_csv(f_info, show_col_types = F) %>% 
    filter(!grepl('^NK', compare_with))  # no NK cells
# rename cell types
for (nm in names(celltype_map)) {
    clone_info$subtype[clone_info$subtype %in% celltype_map[[nm]]] <- nm
    clone_info$compare_with[clone_info$compare_with %in% celltype_map[[nm]]] <- nm
}
# add cell state info
clone_info$cell_state <- 'unknown'
for (nm in names(cell_state_map)) {
    clone_info$cell_state[clone_info$compare_with %in% cell_state_map[[nm]]] <- nm
}
clone_info$cell_state[clone_info$cell_state == 'cytotoxic'] <- 'other cytotoxic'
loginfo('%g records', nrow(clone_info))

2024-02-05 18:00:08 INFO::256399 records


In [45]:
# calculate median score & define as A/B-share/specific
df <- clone_info %>% 
    mutate(rep_ctype = if_else(subtype == 'CD8_CX3CR1', 'A', 'B')) %>% 
    unite(col = 'cell_group', rep_ctype, clone_cat2a, sep = '-', remove = F) %>% 
    summarise(.by = c('sample', 'patient', 'sample_type', 'cell_state', 'cell_group', 'clone_cat2a'),
              exhau_score = median(exhau),
              cyto_score = median(cyto)) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post'))
df %>% write_tsv(str_glue('{outdir}/fig5c-cyto_exhau_score-cell_state.tsv'))
loginfo('%g records after summrizing', nrow(df))

2024-02-05 18:00:50 INFO::these clinial info will be added: response
2024-02-05 18:00:50 INFO::1183 records after summrizing


In [ ]:
# plot
ylab_map <- list('cyto_score' = 'Cytotoxicity score', 'exhau_score' = 'Exhaustion score')
df <- read_tsv(str_glue('{outdir}/fig5c-cyto_exhau_score-cell_state.tsv'), show_col_types = F) %>%
    mutate(cell_state = factor(cell_state, levels = c('other cytotoxic', 'exhausted', 'dying', 'others')),
           cell_group = factor(cell_group, levels = group_order))
p_ls <- lapply(X = names(ylab_map), FUN = function(ycol) {
    ggplot(data = df, aes(x = cell_group, y = .data[[ycol]])) +
    geom_boxplot(aes(fill = clone_cat2a), outlier.shape = NA) +
    # geom_jitter(size = 1.5, alpha = 0.5, width = 0.25) +
    facet_wrap(~ cell_state, nrow = 1) +
    stat_compare_means(comparisons = comp_ls) +
    labs(y = ylab_map[[ycol]], fill = 'TCR Shared Type') +
    theme(axis.text.x = element_text(angle = 45, hjust = .95), axis.title.x = element_blank())
})
ggsave(filename = str_glue('{outdir}/fig5c-box_cyto_exhau_score-a_b_share.pdf'), 
       plot = wrap_plots(p_ls, ncol = 1, guides = 'collect'), width = 14, height = 9)

## 5d: experimental, CXCL13 CD8 in CD8 total

In [ ]:
raw_file <- '/nfs_beijing_td/nfsshare/nfshome/zhe/Ana/240110_ESCC_externalData/staining_raw/1-8-MERGE_cell_seg_data.txt'
sample_file <- '/nfs_beijing_td/nfsshare/nfshome/zhe/Ana/240110_ESCC_externalData/staining_raw/1-8-过滤后保留的MSI视野.xlsx'
cx3cl1_staining_file <- '/nfs_beijing_td/nfsshare/nfshome/zhe/Ana/240110_ESCC_externalData/staining_1_CX3CL1.tsv'
patient_file <- '/nfs_beijing_td/nfsshare/nfshome/zhe/Ana/240110_ESCC_externalData/staining_1-8_patientTbl.tsv'

In [ ]:
df_pat <- read_tsv(patient_file, show_col_types = F) %>%
    mutate(RVT = as.numeric(gsub('%', '', RVT)) / 100)

In [ ]:
valid_samples <- readxl::read_xlsx(sample_file, sheet = 1) %>%
    pull('Sample Name')
df_raw <- read_tsv(raw_file, show_col_types = F) %>% 
    filter(`Sample Name` %in% valid_samples)
colnames(df_raw) <- gsub(" \\(Normalized Counts, Total Weighting\\)", "", colnames(df_raw))
dim(df_raw)

[1] 1157967     141

In [ ]:
# Use sub to extract the number between '-' and '_'
df_raw$exp_id <- sub(".*-(\\d+)_.*", "\\1", df_raw$'Sample Name')
df_raw$code_id = df_pat$code_id[match(df_raw$exp_id, df_pat$exp_id)]

# #constructure raw1 measurement table
# raw1_measurements=colnames(df_raw)[17:133]
# measurement_df1=label_processing(raw1_measurements)

In [ ]:
col_name_map <- c(
    sample_id="Sample Name",
    CX3CR1_total = "Membrane CX3CR1 (Opal 570) Total",
    CX3CR1_mean = "Membrane CX3CR1 (Opal 570) Mean",
    CD8_total = "Membrane CD8 (Opal 620) Total",
    CD8_mean = "Membrane CD8 (Opal 620) Mean",
    CX3CL1_mean = "Membrane CX3CL1 (Opal 540) Mean",
    LYVE1_mean = "Membrane LYVE1 (Opal 690) Mean",
    CD31_mean = "Membrane CD31 (Opal 520) Mean"
)
df <- df_raw %>% 
    rename(all_of(col_name_map)) %>% 
    select(all_of(c('code_id', 'exp_id', names(col_name_map)))) %>% 
    filter(! (code_id %in% c(136, 137)))
df %>% dim

[1] 1157967      10

In [ ]:
CD8_tot_thresh=40
CX3CR1_tot_thresh=40
df_stat <- df %>%
    summarise(
        CX3CR1_CD8_in_CD8_total= sum(CX3CR1_total>CX3CR1_tot_thresh & CD8_total>CD8_tot_thresh)/sum(CD8_total>CD8_tot_thresh),
        CX3CR1_CD8_in_CD8_mean= sum(CX3CR1_mean>CX3CR1_mean_thresh & CD8_mean>CD8_mean_thresh)/sum(CD8_mean>CD8_mean_thresh),
        CX3CR1_CD8_in_all = mean(CX3CR1_mean>CX3CR1_mean_thresh & CD8_mean>CD8_mean_thresh),
        CX3CL1_LYVE1_in_LYVE1_mean= sum(CX3CL1_mean>CX3CL1_Memmean_thresh & LYVE1_mean>LYVE1_mean_thresh )/sum(LYVE1_mean>LYVE1_mean_thresh),
        CX3CL1_CD31_in_CD31_meann= sum(CX3CL1_mean>CX3CL1_Memmean_thresh & CD31_mean>CD31_mean_thresh )/sum(CD31_mean>CD31_mean_thresh),
        CX3CL1_CD31_in_all= mean(CX3CL1_mean>CX3CL1_Memmean_thresh &CD31_mean>CD31_mean_thresh ),
        CX3CL1_CD8_in_CD8_mean= sum(CX3CL1_mean>0.3 & CD8_mean>CD8_mean_thresh)/sum(CD8_mean>CD8_mean_thresh),
        CX3CL1_CX3CR1_CD8_in_CD8_mean= sum(CX3CL1_mean>0.2 & CX3CR1_mean>CX3CR1_mean_thresh & CD8_mean>CD8_mean_thresh)/sum(CD8_mean>CD8_mean_thresh),
        .by=c(code_id, exp_id, sample_id)
    ) %>%
    merge(df_pat, by = c('code_id', 'exp_id'))
code_order <- df_stat %>%
    summarise(rank_val = median(CX3CR1_CD8_in_CD8_total), .by = code_id) %>%
    arrange(-rank_val) %>%
    pull(code_id)
# code_order <- c('142', '140', '139', '145', '143', '144', '138', '141')
df_stat <- df_stat %>% mutate(code_id = factor(code_id, levels = code_order))

p <- ggplot(df_stat, aes(x=code_id,y=CX3CR1_CD8_in_CD8_total, color=RVT)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(position=position_jitter(0.2),aes(fill=RVT)) +
    labs(
        # title=str_glue('CXCL13_all_thresh={CXCL13_all_thresh},CD8_tot_thresh={CD8_tot_thresh}')
        x = 'Patient ID',
        y = 'CX3CR1+ CD8 fraction in total CD8 cells'
    ) +
    theme_classic() + 
    theme(axis.text.x = element_text(angle = 60, vjust = 1, hjust=1))
ggsave(filename = str_glue('../immunity-figs/fig5/fig5d-box-rvt.pdf'), width = 4, height = 4)
a <- df_stat %>% 
    filter(code_id %in% c('130', '135', '129', '134')) %>% 
    pull(CX3CR1_CD8_in_CD8_total)
b <- df_stat %>% 
    filter(code_id %in% c('128', '131', '132', '133')) %>% 
    pull(CX3CR1_CD8_in_CD8_total)
wilcox.test(a, b)


	Wilcoxon rank sum test with continuity correction

data:  a and b
W = 2132, p-value = 1.186e-07
alternative hypothesis: true location shift is not equal to 0


# previous

## clone fraction: shared with CD8_CX3CR1 vs shared with non-CD8_CX3CR1: 5d

- sharing at sample level
- re-calculate clone fraction (based on RNA & TCR data)
- exclude CD8_CX3CR1 when plot

In [48]:
f_clone_info <- '../../stage4/a05_clone_share/clone_share_old/clone_sharing_info_per_cell.csv'
comp_ls <- list(
    c('CD8_CX3CR1_shared', 'otherT_shared'), 
    c('otherT_shared', 'specific'), 
    c('CD8_CX3CR1_shared', 'specific')
)
share_type_label_map <- c(
    'CD8_CX3CR1_shared' = 'shared with\nCD8_CX3CR1',
    'otherT_shared' = 'shared with\nnon-CD8_CX3CR1', 
    'specific' = 'unshared'
)

In [51]:
# re-calculate clone fraction
clone_frac <- read_csv(f_clone_info, show_col_types = F)  %>%
    add_count(sample, clonotype, name = 'n_cell_per_sample_clone') %>% 
    add_count(sample, name = 'n_cell_per_sample') %>% 
    select(sample, patient, sample_type, subtype, clonotype, share_type, n_cell_per_sample_clone, n_cell_per_sample) %>% 
    distinct() %>% 
    mutate(clone_frac_new = n_cell_per_sample_clone / n_cell_per_sample) %>%
    filter(subtype != 'CD8_CX3CR1') %>% # no CD8_CX3CR1
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post'))

# rename cell types
for (nm in names(celltype_map)) {
    clone_frac$subtype[clone_frac$subtype %in% celltype_map[[nm]]] <- nm
}
# add cell state info
clone_frac$cell_state <- 'unknown'
for (nm in names(cell_state_map)) {
    clone_frac$cell_state[clone_frac$subtype %in% cell_state_map[[nm]]] <- nm
}
clone_frac$cell_state[clone_frac$cell_state == 'cytotoxic'] <- 'other cytotoxic'

clone_frac %>% write_tsv(str_glue('{outdir}/fig5d-tcr_share_inter_subtype-clone_frac_new.tsv'), quote = 'needed')
loginfo('%g records of clonotypes in each sample, each subtype', nrow(clone_frac))

2024-02-05 18:06:34 INFO::131179 records of clonotypes in each sample, each subtype


In [53]:
# plot
p <- read_tsv(str_glue('{outdir}/fig5d-tcr_share_inter_subtype-clone_frac_new.tsv'), show_col_types = F) %>% 
    mutate(cell_state = factor(cell_state, levels = c('other cytotoxic', 'exhausted', 'dying', 'others')),
           share_type = factor(share_type, levels = names(share_type_label_map))) %>%
    filter(!grepl('^NK', subtype)) %>%
    ggviolin(x = 'share_type', y = 'clone_frac_new', fill = 'share_type', 
             facet.by = 'cell_state', ncol = 9, add = 'boxplot') + 
    stat_compare_means(comparisons = comp_ls) +
    scale_fill_jco(breaks = names(share_type_label_map), labels = share_type_label_map) +
    scale_x_discrete(labels = share_type_label_map) +
    scale_y_continuous(trans = 'log10') +  # log10 won't influence wilcoxon test p value
    labs(y = 'Clonotype fraction at sample level\n(RNA & TCR data)', fill = 'TCR sharing type') +
    theme(axis.text.x = element_text(angle = 45, hjust = 0.95), axis.title.x = element_blank())
ggsave(filename = str_glue('{outdir}/fig5d-vln_tcr_share_inter_subtype-clone_frac_new.pdf'), 
       plot = p, width = 14, height = 6)

## CX3CL1 expression: 5e-f

- date: 2024-01-04

### dotplot in whole data: 5e

In [9]:
scrna <- readRDS('../../stage4/a01_data/seuobj/whole.rds')

In [12]:
scrna$celltype <- scrna$cell_type_correct
scrna$celltype[scrna$cell_type_correct %in% c('B', 'Plasma')] <- 'B & Plasma cells'
scrna$celltype[scrna$cell_type_correct %in% c('T')] <- 'T cells'
table(scrna$celltype)


B & Plasma cells      Endothelial       Epithelial       Fibroblast 
          103212            44055            75040           111164 
         Myeloid          T cells 
          114636           302025 

In [14]:
p <- Seurat::DotPlot(scrna, features = 'CX3CL1', group.by = 'celltype') +
    scale_color_gsea() +
    theme(axis.title = element_blank())
ggsave(filename = str_glue('{outdir}/fig5e-dot_exprs-cx3cl1-whole.pdf'), plot = p, width = 3.5, height = 4)

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


### pbulk in sample: 5f

In [37]:
f_exprs <- '../../stage4/a04_cx3cl1/tme_exprs/cx3cl1_tme_exprs.tsv'

In [38]:
# add clin info
df <- read_tsv(f_exprs, show_col_types = F) %>%
    separate(col = 'sample', into = c('patient', 'sample_type'), sep = '-', remove = F) %>% 
    mutate(cx3cl1_exprs_log = log1p(cx3cl1_exprs_unlog)) %>% 
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post'))

df %>% write_tsv(str_glue('{outdir}/fig5f-pbulk_exprs-cx3cl1_in_TME.tsv'))

2024-02-05 17:19:07 INFO::these clinial info will be added: response


In [40]:
p <- read_tsv(str_glue('{outdir}/fig5f-pbulk_exprs-cx3cl1_in_TME.tsv'), show_col_types = F) %>% 
    filter(!is.na(.data[[gp]])) %>%
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'cx3cl1_exprs_log', pt_fill = gp, facet_by = NULL,
                      xorder = comb_order, fill_order = gp_lvls[[gp]], xangle = 60) + 
    stat_compare_means(comparisons = gp_comp_map[[gp]]) +
    labs(y = 'CX3CL1 expression in sample', fill = 'Response')
ggsave(filename = str_glue('{outdir}/fig5f-box_exprs-cx3cl1_pbulk_in_TME.pdf'), plot = p, height = 5, width = 4)

## todos